PART- A

In [ ]:
!pip install langchain chromadb
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 5.9 MB/s eta 0:0

In [ ]:
!pip install langsmith==0.1.125


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.147
    Uninstalling langsmith-0.1.147:
      Successfully uninstalled langsmith-0.1.147


In [ ]:
!pip install -U langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [ ]:
import os
import glob
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
os.environ["OPENAI_API_KEY"] = "your-api-key"
FILE_PATH = "generate.py"

def load_single_file(file_path: str):
    if not os.path.isfile(file_path):
        raise ValueError(f"No file found at {file_path}")
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
        return [Document(page_content=content, metadata={"source": file_path})]

code_docs = load_single_file(FILE_PATH)

if not code_docs:
    raise ValueError("No code files found. Please ensure the specified file exists.")

# Spliting code into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len
)
split_docs = text_splitter.split_documents(code_docs)

# embeddings
embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])

# Chroma vectorstore
db = Chroma.from_documents(split_docs, embeddings, collection_name="codebase")

# retrieval-based QA system
retriever = db.as_retriever(search_kwargs={"k": 3})
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(
        openai_api_key=os.environ["OPENAI_API_KEY"],
        model="gpt-3.5-turbo",
        temperature=0
    ),
    retriever=retriever,
    chain_type="stuff"
)

query = "What does the function 'generate_images' do in my codebase?"
response = qa_chain.run(query)
print("Query:", query)
print("Answer:", response)


Query: What does the function 'generate_images' do in my codebase?
Answer: The `generate_images` function in your codebase is a command-line interface function that uses Click to define command-line options for generating images using a pretrained StyleGAN2 model. This function takes in parameters such as the network pickle filename, random seeds, truncation psi, noise mode, output directory, class label, and projected W file. It allows you to generate images with different configurations like curated MetFaces images, uncurated MetFaces images, class conditional CIFAR-10 images, and rendering an image from projected W.


In [ ]:
query = "What are these documents about?"
answer = qa_chain.run(query)
print(answer)

The code snippets provided are related to generating images using a pre-trained StyleGAN model. The code involves loading a pre-trained network, specifying labels for conditional generation, generating images based on random seeds or projected W values, and saving the generated images to an output directory. The code uses PyTorch and PIL libraries for image processing and manipulation.


PART- B

In [ ]:
!pip install langchain pypdf tiktoken chromadb openai

In [ ]:
import os
import glob
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain.document_loaders import PyPDFLoader

os.environ["OPENAI_API_KEY"] = "your-api-key"

pdf_path = "Part3_knowledge base.pdf"
if not os.path.exists(pdf_path):
    raise ValueError(f"File {pdf_path} does not exist.")

# Loading PDF
loader = PyPDFLoader(pdf_path)
docs = loader.load()

if not docs:
    raise ValueError("No content extracted from the PDF file.")

# Spliting into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100, length_function=len)
split_docs = text_splitter.split_documents(docs)

# embeddings
embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])

db = Chroma.from_documents(split_docs, embedding=embeddings, collection_name="chat_docs")
retriever = db.as_retriever(search_kwargs={"k": 3})

# memory object
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Conversational Retrieval Chain
chat = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
    retriever=retriever,
    memory=memory
)

# chatting
user_questions = [
    "Can you summarize the main topic of this document?",
    "Give me more details on the methodology mentioned.",
    "Are there any notable results or conclusions mentioned?"
]

for question in user_questions:
    response = chat({"question": question})
    print("User:", question)
    print("AI:", response["answer"])
    print()

# Summarizing entire conversation
conversation_messages = memory.load_memory_variables({})["chat_history"]

conversation_text = ""
for msg in conversation_messages:
    role = "User" if msg.type == "human" else "AI"
    conversation_text += f"{role}: {msg.content}\n"

summary_prompt = PromptTemplate(
    input_variables=["conversation"],
    template="""
Summarize the entire conversation below into a concise summary:

Conversation:
{conversation}
"""
)

summary_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
summary = summary_llm([HumanMessage(content=summary_prompt.format(conversation=conversation_text))])
print("Summary of the conversation:")
print(summary.content)


User: Can you summarize the main topic of this document?
AI: The document discusses Meta's financial results and highlights from the past year, emphasizing their focus on operating discipline, product priorities, and advertising performance. It also mentions their plans for 2024, including advancements in AI and Reality Labs. Additionally, it addresses potential risks and uncertainties related to their business operations, such as geopolitical events, competition, privacy concerns, and regulatory issues. The document also provides information about an upcoming conference call to discuss the results and where to access related materials.

User: Give me more details on the methodology mentioned.
AI: The document mentions the use of non-GAAP financial measures such as revenue excluding foreign exchange effect, advertising revenue excluding foreign exchange effect, and free cash flow. These measures are used to supplement the company's financial statements prepared in accordance with GAAP.